<a href="https://colab.research.google.com/github/rodrigoataidealves/TCC_MBA_USP/blob/main/Codigo_python_TCC_MBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# -*- coding: utf-8 -*-
# SICONFI API → RREO – Anexo 01 • 1º bimestre/2025 • Todos os municípios de MG
# Gera DataFrame `receitas_1bi_2025`, salva em Excel e disponibiliza o download no Colab.

import time
import requests
import pandas as pd
from urllib.parse import urlencode

# ===========================
# Parâmetros principais
# ===========================
UF_ALVO = "MG"
ANO = 2025
BIMESTRE = 1
TIPO = "RREO"
ANEXOS_POSSIVEIS = ["RREO-Anexo 01", "RREO-Anexo 1"]  # variações comuns no endpoint

# Endpoints base (API pública SICONFI/ORDS)
URL_ENTES = "https://apidatalake.tesouro.gov.br/ords/siconfi/tt/entes"
URL_RREO  = "https://apidatalake.tesouro.gov.br/ords/siconfi/tt/rreo"

# Nome do arquivo Excel (usei o que você costuma preferir; troque se quiser)
EXCEL_NAME = "DFLUX_Tabela_ManualPDG.xlsx"

# ===========================
# Utilitários
# ===========================
def fetch_all(url, params=None, page_size=5000, timeout=40):
    """
    Faz paginação ORDS e retorna todos os 'items'.
    """
    params = params.copy() if params else {}
    items = []
    offset = 0
    while True:
        qp = params.copy()
        qp.update({"limit": page_size, "offset": offset})
        resp = requests.get(url, params=qp, timeout=timeout)
        if resp.status_code != 200:
            raise RuntimeError(f"Falha HTTP {resp.status_code} em {resp.url}\n{resp.text[:500]}")
        data = resp.json()
        batch = data.get("items", [])
        items.extend(batch)
        if not data.get("hasMore"):
            break
        offset = data.get("offset", 0) + data.get("limit", len(batch))
    return items

def get_entes_mg():
    """
    Baixa o cadastro de entes e filtra Municípios de MG.
    """
    entes = pd.DataFrame(fetch_all(URL_ENTES))
    entes.columns = [c.lower() for c in entes.columns]

    # Checagem de campos esperados
    if "uf" not in entes.columns or "esfera" not in entes.columns:
        raise RuntimeError("O endpoint /entes não retornou 'uf' e/ou 'esfera'.")

    cod_col = "cod_ibge" if "cod_ibge" in entes.columns else ("id_ente" if "id_ente" in entes.columns else None)
    if not cod_col:
        raise RuntimeError("Não encontrei 'cod_ibge' nem 'id_ente' no cadastro de entes.")

    nome_col = "ente" if "ente" in entes.columns else ("no_municipio" if "no_municipio" in entes.columns else None)
    if not nome_col:
        nome_col = "municipio"
        entes[nome_col] = entes.get("no_ente", entes.get("nm_ente", ""))

    entes_mg = entes[(entes["uf"].str.upper() == UF_ALVO) & (entes["esfera"].str.upper() == "M")].copy()
    entes_mg = entes_mg[[cod_col, nome_col, "uf"]].rename(columns={cod_col: "cod_ibge", nome_col: "municipio"})
    entes_mg["cod_ibge"] = entes_mg["cod_ibge"].astype(str)
    return entes_mg

def chama_rreo_um_ente(cod_ibge, anexo_nome):
    """
    Chama o RREO Anexo 01 para 1 ente e retorna a(s) linha(s) 'RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS)'
    já pivoteada(s) em colunas (ex.: Previsão Inicial, No Bimestre, Até o Bimestre etc.).
    """
    params = {
        "an_exercicio": ANO,
        "nr_periodo": BIMESTRE,
        "co_tipo_demonstrativo": TIPO,
        "no_anexo": anexo_nome,
        "id_ente": cod_ibge
    }
    resp = requests.get(URL_RREO, params=params, timeout=40)
    if resp.status_code != 200:
        return pd.DataFrame()
    data = resp.json()
    items = data.get("items", [])
    if not items:
        return pd.DataFrame()

    df = pd.DataFrame(items)
    df.columns = [c.lower() for c in df.columns]

    campo_rotulo = next((c for c in ["rotulo", "ds_conta", "conta", "no_conta", "descricao"] if c in df.columns), None)
    campo_quadro = next((c for c in ["quadro", "ds_quadro", "no_quadro"] if c in df.columns), None)
    campo_coluna = next((c for c in ["coluna", "ds_coluna", "no_coluna"] if c in df.columns), None)
    campo_valor = next((c for c in ["valor", "vl", "vlr", "vl_valor"] if c in df.columns), None)

    if not (campo_rotulo and campo_coluna and campo_valor):
        return pd.DataFrame()

    # Filtro: linha de Receitas (Exceto Intra) no quadro de Receitas Orçamentárias
    mask_exceto = df[campo_rotulo].astype(str).str.upper().str.contains("EXCETO INTRA")
    mask_quadro = True if not campo_quadro else df[campo_quadro].astype(str).str.upper().str.contains("RECEITAS ORÇAMENTÁRIAS")
    df_exceto = df[mask_exceto & mask_quadro].copy()
    if df_exceto.empty:
        return pd.DataFrame()

    # Converte valores numéricos e pivota
    df_exceto[campo_valor] = pd.to_numeric(df_exceto[campo_valor], errors="coerce")
    tabela = (
        df_exceto[[campo_rotulo, campo_coluna, campo_valor]]
        .pivot_table(index=campo_rotulo, columns=campo_coluna, values=campo_valor, aggfunc="sum")
        .reset_index(drop=False)
    )
    return tabela

# ===========================
# Execução
# ===========================
print("1) Carregando municípios de MG (API /entes)…")
entes_mg = get_entes_mg()
print(f"   • Municípios encontrados: {len(entes_mg)}")

linhas = []
falhas = []

print("2) Consultando RREO – Anexo 01 • 1º bimestre/2025 por município… (pode levar alguns minutos)")
for i, row in entes_mg.iterrows():
    cod = row["cod_ibge"]
    mun = row["municipio"]
    ok = False
    for anexo in ANEXOS_POSSIVEIS:
        df_ente = chama_rreo_um_ente(cod, anexo)
        if not df_ente.empty:
            df_ente.insert(0, "cod_ibge", cod)
            df_ente.insert(1, "municipio", mun)
            df_ente.insert(2, "uf", UF_ALVO)
            df_ente.insert(3, "ano", ANO)
            df_ente.insert(4, "bimestre", BIMESTRE)
            linhas.append(df_ente)
            ok = True
            break
    if not ok:
        falhas.append((cod, mun))
    if (i + 1) % 50 == 0:
        time.sleep(0.5)  # cortesia com o serviço

# Consolida no DataFrame solicitado
if linhas:
    receitas_1bi_2025 = pd.concat(linhas, ignore_index=True)
else:
    receitas_1bi_2025 = pd.DataFrame()

print("\n3) Resumo da coleta")
print(f"   • Municípios com retorno: {receitas_1bi_2025['municipio'].nunique() if not receitas_1bi_2025.empty else 0}")


1) Carregando municípios de MG (API /entes)…
   • Municípios encontrados: 853
2) Consultando RREO – Anexo 01 • 1º bimestre/2025 por município… (pode levar alguns minutos)

3) Resumo da coleta
   • Municípios com retorno: 0
